<a href="https://colab.research.google.com/github/curimm/JavaScript/blob/main/202306_08)DataCampus_PJ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

기사의 주제 찾기 및 내년의 핫토픽 예측(회귀)

전처리 : 2000년부터 2018년까지 자료(2000~2010)

웹크롤링(영어논문, Abstract)>엑셀>전처리
논문의 문서별 단어토큰화:['토픽', '멋짐']
정규표현식:문자열에 대해 원하는 검색패턴 지정
불용어처리:추가로 작성하여 몇개의 단어를 추가로 제거
3. 카운트 벡터화(doc2bow) : Gensim의 명령어
예) 특성집합=['one', 'two', 'three', 'four', 'five']
예) 주어진 문서의 토큰화 결과=['two', 'two', 'four']
예) 변환된 카운트벡터 결과=[0, 2, 0, 1, 0]
4. 토픽모델링(Gensim의 LdaModel사용)
5. 혼란도와 응집도를 이용하여 토픽 개수의 최적값 선택
6. 최적의 토픽값을 이용하여 토픽을 7개로 결정
7. 토픽별 최빈 단어 10개를 활용하여 주제 찾기
8. 선형회귀(X:year, Y:Topic 7개의 해당연도별 비중)를 이용하여 내년의 핫토픽을 예측


In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request
import requests
import csv
import lxml                                     # API로 Abstract 자료를 받은 부분이 XML 자료임.
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
dfa = pd.read_excel("c:\workspace\environment.xls")   # 65535 row 이상은 불러들이지 못함
dfa


df=dfa[['Year', 'Content']]     # 분석의 대상이 되는 "연도(year)"에 따른 "Abstract"를 DataFrame 구조로 읽어옴
df


FileNotFoundError: [Errno 2] No such file or directory: 'c:\\workspace\\environment.xls'

In [ ]:

df = df.dropna()                             # Abstract의 결측치가 있어서 이를 제거함
df

In [ ]:

df.Content[0]                                            #첫번째 Abstract를 제대로 읽어왔는지 확인함


In [ ]:

df.Content[1]                                            #두번째 Abstract를 제대로 읽어왔는지 확인함



In [6]:
# In[70]:

cachedStopWords = stopwords.words("english")     # nltk에서 제공한 불용어 사용
RegTok = RegexpTokenizer("[\w']{2,}")            # 정규표현식으로 토크나이저를 정의. 빈칸없는 단어로 3글자 이상인 단어만 토큰화시킴
english_stops = set(stopwords.words('english'))  # 금지어가 반복되지 않게 set으로 변환
my_stopword=['\r','대한','위해', '환경부', '2016', '는', '등을', '따라', '위한', '대해', '밝혔다', '했다',
            '경우', '한다', '2013년', '2012년', '또한', '관련', '이번', '통해', '관련', '있는', '것으로',
             '나타났다', '09', '가장', '지난', '지역', '기자', '함께', '예정이다', '있도록', '무단전재',
            'kr', 'co', '받을', '변형', '기사에', '알려드립니다', '금합니다', '무단배포를', '때문에', '것이',
            '말했다', '것이다', '있어', '많은', '것은', '아니라', '계획이다', '사업을', '2015년', '연간',
            'ecolaw', '환경법률신문', '서울시', '서울시는', '처음', '으로', '20일', '따르면', '향으로', '각종',
            '드는', '1백', '결정했다', '인천지방해양수산청에', '해양수산부는', '이번에', '따른','된다', '전례없이',
            'www', 'http', '2011', '있으며', '다양한', '행사는', '쉽게', '환경일보', '하는', '같은', '에너지데일리',
            '이를', '있으며', '기술', '08', '발생하는', '저작권은', '있음을', '이를', '있으며', '에코저널', '2017',
            '에코저널에', '환경부는', '2012', '장관', '대상으로', '나갈', '올해', '환경일보', '결과', '평균', '비해',
            '전국', '높은', '10', '12', '이상', '이에', '현재', '그러나', '인한', '있다고', '인해', '최근', '이라고',
            '사업', '관계자는', '07', '사업은', '이날', '오는', '주제로', '대상으로', '오후', '가운데', '한편',
            '2015', '2013', '향후', '환경', '조사', '특히', '전체', '각각', '11', '대비', '홈페이지', '분야', '에서',
            '직접', '2014년', '저작권자', 'webmaster', '다른', '이러한', '있을', '있고', '매우', '것을', '새로',
            'ecojournal', '등의', '또는', '환경법률신문에', 'kr이', '의견을', '진행된다', '2018', '방안을', '에코타임스',
            'ecotiger', '2016년', '재배포금지', '저작권자', '피해를', '피해', '베트남', '라고', '있다는', '하지만',
            '없는', '한다고', '교수는', '강조했다', '위해서는', '않고', '2009년', '인증을', '진행항다', '공유하고',
            '무단', '최우수', '기존', '시는', '내년', '지난해', '이후', '모두', '이는', '발생한', '보다', '등에',
             '에코타임스에', '대해서는', '해당', '통한', '추진할', '거쳐', 'daum', 'net', '만들기', '좋은',
            '시민들의', '10월', '기술을', '시민들이', '우리', '명이', '등이', '아름다운', '동안', '제품', '2014',
             '개발', '어길', '이정성', 'jungsung|', '인근', '진행됐다', '기준', '영향을', '확인됐다', '30',
             '20', '증가한', '2017년', '2010', '하고', '않은', '일부', '않는', '문제가', '것이라고', '모든', '관한',
            '주요', '실시하고', '경기도는', '적극', '일자리', '정부는', '평가를', '이용해', '기반', '많이', '원장',
            '자세한', '연구를', '의원은', '그는', '라며', '지적했다', '필요하다', '한다는', '있지만', '설명했다',
            '주장했다', '발생', '주변', '기대된다', '방침이다', '물론', '지속적으로', '개최한다', '지속가능한',
            '이어', '개최했다', '연구', '기준을', '결과를', '낮은', '실시한', '절약', '국립공원', '피해가', '없다',
            '만큼', '있었다', '때문이다', '이미', '않아', '대응', '방안', '당부했다', '만들어', '새로운', '지원한다',
            '건강한', '전남',' 제주', '강원도', '추진', '있게', '필요한', '운영', '기여할', '세계', '국내', '글로벌',
            '앞으로도', '어린이', '실시한다', '관내', '서울', '11월', '참석한', '9월', '계기가', '경기도', '추진하고',
            '원을', '선정됐다', '수돗물을', '차량', '15', '태풍', '문제를', '제대로', '해야', '과정에서', '우리나라',
            '함께하는', '금번', '중요한', '이용한', '사용', '정보를', '서비스를']
# Abstract의 앞부분에 집중적으로 반복되는 주제와 상관없는 단어(연구의 목적)를
 ## 개인적으로 금지어로 정함

def tokenizer(text):
    tokens = RegTok.tokenize(text.lower())
    words = [word for word in tokens if (word not in english_stops) and len(word)>1 and (word not in my_stopword)]
    return words

texts = [tokenizer(news) for news in df.Content]    # 토큰화가 잘 되어 있는 것을 확인함.
texts

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:

# Gensim은 토큰화 결과로부터 토큰과 gensim 모듈이 내부적으로 사용하는 id를 매칭하는 사전을 만든다.
# 이렇게 사전을 만들게 하는 클래스가 Dictionary이다. (사전생성)
# filter_extremes() 메서드 : 빈도수가 높은 순으로 지정된 수의 단어를 특성으로 선택한다.
# no_below=5 : 노출된 단어 수가 5번 미만의 단어는 버린다.
# no_above=0.5 : 노출된 단어 비중이 50%를 넘는 단어는 버린다.

# In[71]:

from gensim.corpora.dictionary import Dictionary        #  gensim을 써서 카운트 벡터로 변환하고, 혼란도와 응집도를 구해서
                                                         ## 토픽의 수를 결정하는데 사용함
dictionary = Dictionary(texts)                           # 토큰화 결과로부터 dictionay 생성
print('#Number of initial unique words in documents:', len(dictionary))

                     # 단어 빈도수가 너무 적거나(5번 미만) 높은 단어(50%를 초과)를 배제하고 특성(단어)을 단어의 빈도 순으로 나타냄.
dictionary.filter_extremes(keep_n=2000, no_below=5, no_above=0.5)     # 단어의 최대수를 2000으로 한정함
print('#Number of unique words after removing rare and common words:', len(dictionary))

corpus = [dictionary.doc2bow(text) for text in texts]       # doc2bow(text) : 카운트 벡터로 변환
print('#Number of unique tokens: %d' % len(dictionary))
print('#Number of documents: %d' % len(corpus))


In [ ]:
from gensim.models import LdaModel

num_topics = 10                 #Topic수는 임의로 10으로 정한 후 혼란도와 응집도를 계산하는데 사용함.
passes = 5                      # passes : 최대로 반복가능한 알고리즘수
get_ipython().run_line_magic('time', 'model = LdaModel(corpus=corpus, id2word=dictionary,                        passes=passes, num_topics=num_topics,                         random_state=7)')


In [ ]:
# print_topics() 메서드 : gensim 제공, 각 토픽의 상위비중단어(비중도 표시함)를 나타냄
# num_words : 각 토픽의 상위 단어 수 지정

# #  코딩 및 결과 도출, 불용어 처리중

In [ ]:
model.print_topics(num_words=10)    # 결과에 의거 불용어 선정 피드백

In [ ]:

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

In [ ]:
# feed thd LDA model in the pyLDAvis instance
lda_viz = gensimvis.prepare(model, corpus, dictionary)
lda_viz


In [ ]:
from gensim.models import CoherenceModel

cm = CoherenceModel(model=model, corpus=corpus, coherence='u_mass')
coherence = cm.get_coherence()
print(coherence)

In [ ]:

def show_coherence(corpus, dictionary, start=3, end=15):
    iter_num = []
    per_value = []
    coh_value = []

    for i in range(start, end + 1):   # range(3, 16) : 3,4,...,15
        model = LdaModel(corpus=corpus, id2word=dictionary,
                 chunksize=1000, num_topics=i,
                 random_state=7)
        iter_num.append(i)
        pv = model.log_perplexity(corpus)
        per_value.append(pv)

        cm = CoherenceModel(model=model, corpus=corpus,
                            coherence='u_mass')
        cv = cm.get_coherence()
        coh_value.append(cv)
        print(f'num_topics: {i}, perplexity: {pv:0.3f}, coherence: {cv:0.3f}')

    plt.plot(iter_num, per_value, 'g-')
    plt.xlabel("num_topics")
    plt.ylabel("perplexity")
    plt.show()

    plt.plot(iter_num, coh_value, 'r--')
    plt.xlabel("num_topics")
    plt.ylabel("coherence")
    plt.show()

show_coherence(corpus, dictionary, start=3, end=15)


In [ ]:

# 혼란도가 낮으면서 응집도가 높은 것을 4로 선택함.
## 사이킷런을 이용한 토픽모델링
from sklearn.feature_extraction.text import CountVectorizer  #LDA는 카운트벡터를 입력으로 사용한다. 카운트벡터 생성과정 중 토큰화과정.

cachedStopWords = stopwords.words("english")
RegTok = RegexpTokenizer("[\w']{2,}")                        # 정규표현식으로 토크나이저를 정의
english_stops = set(stopwords.words('english'))              # 금지어를 반복되지 않게 set으로 변환
my_stopword=['\r','대한','위해', '환경부', '2016', '는', '등을', '따라', '위한', '대해', '밝혔다', '했다',
            '경우', '한다', '2013년', '2012년', '또한', '관련', '이번', '통해', '관련', '있는', '것으로',
             '나타났다', '09', '가장', '지난', '지역', '기자', '함께', '예정이다', '있도록', '무단전재',
            'kr', 'co', '받을', '변형', '기사에', '알려드립니다', '금합니다', '무단배포를', '때문에', '것이',
            '말했다', '것이다', '있어', '많은', '것은', '아니라', '계획이다', '사업을', '2015년', '연간',
            'ecolaw', '환경법률신문', '서울시', '서울시는', '처음', '으로', '20일', '따르면', '향으로', '각종',
            '드는', '1백', '결정했다', '인천지방해양수산청에', '해양수산부는', '이번에', '따른','된다', '전례없이',
            'www', 'http', '2011', '있으며', '다양한', '행사는', '쉽게', '환경일보', '하는', '같은', '에너지데일리',
            '이를', '있으며', '기술', '08', '발생하는', '저작권은', '있음을', '이를', '있으며', '에코저널', '2017',
            '에코저널에', '환경부는', '2012', '장관', '대상으로', '나갈', '올해', '환경일보', '결과', '평균', '비해',
            '전국', '높은', '10', '12', '이상', '이에', '현재', '그러나', '인한', '있다고', '인해', '최근', '이라고',
            '사업', '관계자는', '07', '사업은', '이날', '오는', '주제로', '대상으로', '오후', '가운데', '한편',
            '2015', '2013', '향후', '환경', '조사', '특히', '전체', '각각', '11', '대비', '홈페이지', '분야', '에서',
            '직접', '2014년', '저작권자', 'webmaster', '다른', '이러한', '있을', '있고', '매우', '것을', '새로',
            'ecojournal', '등의', '또는', '환경법률신문에', 'kr이', '의견을', '진행된다', '2018', '방안을', '에코타임스',
            'ecotiger', '2016년', '재배포금지', '저작권자', '피해를', '피해', '베트남', '라고', '있다는', '하지만',
            '없는', '한다고', '교수는', '강조했다', '위해서는', '않고', '2009년', '인증을', '진행항다', '공유하고',
            '무단', '최우수', '기존', '시는', '내년', '지난해', '이후', '모두', '이는', '발생한', '보다', '등에',
             '에코타임스에', '대해서는', '해당', '통한', '추진할', '거쳐', 'daum', 'net', '만들기', '좋은',
            '시민들의', '10월', '기술을', '시민들이', '우리', '명이', '등이', '아름다운', '동안', '제품', '2014',
             '개발', '어길', '이정성', 'jungsung|', '인근', '진행됐다', '기준', '영향을', '확인됐다', '30',
             '20', '증가한', '2017년', '2010', '하고', '않은', '일부', '않는', '문제가', '것이라고', '모든', '관한',
            '주요', '실시하고', '경기도는', '적극', '일자리', '정부는', '평가를', '이용해', '기반', '많이', '원장',
            '자세한', '연구를', '의원은', '그는', '라며', '지적했다', '필요하다', '한다는', '있지만', '설명했다',
            '주장했다', '발생', '주변', '기대된다', '방침이다', '물론', '지속적으로', '개최한다', '지속가능한',
            '이어', '개최했다', '연구', '기준을', '결과를', '낮은', '실시한', '절약', '국립공원', '피해가', '없다',
            '만큼', '있었다', '때문이다', '이미', '않아', '대응', '방안', '당부했다', '만들어', '새로운', '지원한다',
            '건강한', '전남',' 제주', '강원도', '추진', '있게', '필요한', '운영', '기여할', '세계', '국내', '글로벌',
            '앞으로도', '어린이', '실시한다', '관내', '서울', '11월', '참석한', '9월', '계기가', '경기도', '추진하고',
            '원을', '선정됐다', '수돗물을', '차량', '15', '태풍', '문제를', '제대로', '해야', '과정에서', '우리나라',
            '함께하는', '금번', '중요한', '이용한', '사용', '정보를', '서비스를']

def tokenizer(text):
    tokens = RegTok.tokenize(text.lower())
    words = [word for word in tokens if (word not in english_stops) and len(word)>1 and (word not in my_stopword)]
    return words

texts = [tokenizer(news) for news in df.Content]          # 토큰화 완료
texts

In [ ]:

from sklearn.feature_extraction.text import CountVectorizer  # LDA는 카운트벡터를 입력으로 사용한다.

cachedStopWords = stopwords.words("english")
RegTok = RegexpTokenizer("[\w']{2,}")                         # 정규표현식으로 토크나이저를 정의
english_stops = set(stopwords.words('english'))               # 반복되지 않게 set으로 변환
my_stopword=['\r','대한','위해', '환경부', '2016', '는', '등을', '따라', '위한', '대해', '밝혔다', '했다',
            '경우', '한다', '2013년', '2012년', '또한', '관련', '이번', '통해', '관련', '있는', '것으로',
             '나타났다', '09', '가장', '지난', '지역', '기자', '함께', '예정이다', '있도록', '무단전재',
            'kr', 'co', '받을', '변형', '기사에', '알려드립니다', '금합니다', '무단배포를', '때문에', '것이',
            '말했다', '것이다', '있어', '많은', '것은', '아니라', '계획이다', '사업을', '2015년', '연간',
            'ecolaw', '환경법률신문', '서울시', '서울시는', '처음', '으로', '20일', '따르면', '향으로', '각종',
            '드는', '1백', '결정했다', '인천지방해양수산청에', '해양수산부는', '이번에', '따른','된다', '전례없이',
            'www', 'http', '2011', '있으며', '다양한', '행사는', '쉽게', '환경일보', '하는', '같은', '에너지데일리',
            '이를', '있으며', '기술', '08', '발생하는', '저작권은', '있음을', '이를', '있으며', '에코저널', '2017',
            '에코저널에', '환경부는', '2012', '장관', '대상으로', '나갈', '올해', '환경일보', '결과', '평균', '비해',
            '전국', '높은', '10', '12', '이상', '이에', '현재', '그러나', '인한', '있다고', '인해', '최근', '이라고',
            '사업', '관계자는', '07', '사업은', '이날', '오는', '주제로', '대상으로', '오후', '가운데', '한편',
            '2015', '2013', '향후', '환경', '조사', '특히', '전체', '각각', '11', '대비', '홈페이지', '분야', '에서',
            '직접', '2014년', '저작권자', 'webmaster', '다른', '이러한', '있을', '있고', '매우', '것을', '새로',
            'ecojournal', '등의', '또는', '환경법률신문에', 'kr이', '의견을', '진행된다', '2018', '방안을', '에코타임스',
            'ecotiger', '2016년', '재배포금지', '저작권자', '피해를', '피해', '베트남', '라고', '있다는', '하지만',
            '없는', '한다고', '교수는', '강조했다', '위해서는', '않고', '2009년', '인증을', '진행항다', '공유하고',
            '무단', '최우수', '기존', '시는', '내년', '지난해', '이후', '모두', '이는', '발생한', '보다', '등에',
             '에코타임스에', '대해서는', '해당', '통한', '추진할', '거쳐', 'daum', 'net', '만들기', '좋은',
            '시민들의', '10월', '기술을', '시민들이', '우리', '명이', '등이', '아름다운', '동안', '제품', '2014',
             '개발', '어길', '이정성', 'jungsung|', '인근', '진행됐다', '기준', '영향을', '확인됐다', '30',
             '20', '증가한', '2017년', '2010', '하고', '않은', '일부', '않는', '문제가', '것이라고', '모든', '관한',
            '주요', '실시하고', '경기도는', '적극', '일자리', '정부는', '평가를', '이용해', '기반', '많이', '원장',
            '자세한', '연구를', '의원은', '그는', '라며', '지적했다', '필요하다', '한다는', '있지만', '설명했다',
            '주장했다', '발생', '주변', '기대된다', '방침이다', '물론', '지속적으로', '개최한다', '지속가능한',
            '이어', '개최했다', '연구', '기준을', '결과를', '낮은', '실시한', '절약', '국립공원', '피해가', '없다',
            '만큼', '있었다', '때문이다', '이미', '않아', '대응', '방안', '당부했다', '만들어', '새로운', '지원한다',
            '건강한', '전남',' 제주', '강원도', '추진', '있게', '필요한', '운영', '기여할', '세계', '국내', '글로벌',
            '앞으로도', '어린이', '실시한다', '관내', '서울', '11월', '참석한', '9월', '계기가', '경기도', '추진하고',
            '원을', '선정됐다', '수돗물을', '차량', '15', '태풍', '문제를', '제대로', '해야', '과정에서', '우리나라',
            '함께하는', '금번', '중요한', '이용한', '사용', '정보를', '서비스를']

def tokenizer(text):
    tokens = RegTok.tokenize(text.lower())
    words = [word for word in tokens if (word not in english_stops) and len(word)>1 and (word not in my_stopword)]
    return words

vec = CountVectorizer(tokenizer=tokenizer,
                      max_df=0.5, min_df = 5,
                      max_features = 2000)                   # 최대 사용 단어수를 2000으로 선택

child_cv = vec.fit_transform(df.Content)                    # 카운트벡터화 완료함. 단어수가 1627임.
print(child_cv.shape)


In [ ]:

from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components = 4,                       # n_components = 4 : 토픽의 수를 4로 고정함
                                n_jobs= -1,
                                random_state=0)

get_ipython().run_line_magic('time', 'child_topics = lda.fit_transform(child_cv)')

def print_top_words(model, feature_names, n_top_words):                # n_top_words=10 : 각각의 토픽에 빈도수가 높은 순으로 단어 10개를 나타냄
    for topic_idx, topic in enumerate(model.components_):
        # enumerate() : 매개변수에 리스트를 넣으면 인텍스와 값을 쌍으로 사용해 반복문을 돌릴 수 있게 해주는 함수
        print("Topic #%d: " % topic_idx, end='')
        print(", ".join([feature_names[i]
                         for i in topic.argsort()[:-n_top_words - 1:-1]]))
                        # topic.argsort()[:-n_top_words - 1:-1] : argsort() 할 때 내림차순으로 정열해라.
print_top_words(lda,vec.get_feature_names(), 10)            # 토픽별 최빈단어순으로 문자를 나타냄.


In [ ]:

trend_data = pd.DataFrame(child_topics, columns=['Topic'+str(i) for i in range(1, 5)])
trend_data


In [ ]:

trend_data = pd.concat([trend_data, df.Year], axis=1)
trend_data


In [ ]:

trend_data = trend_data.dropna()   # 결측치가 있는 데이터는 행기준으로 통째로 없애라
trend_data



In [ ]:

trend = trend_data.groupby(['Year']).mean()
trend.iloc[:, :]  # 모든 행의 인덱스, 모든 열의 인덱스에 해당하는 데이터를 모두 불러와라.


In [ ]:
import matplotlib
import matplotlib.pyplot as plt
# %matplotlib inline

fig, axes = plt.subplots(2, 2, sharex='col', figsize=(12, 16))

  # figsize=(12, 16) : 가로 12인치, 세로 16인치의 도화지에
  # 2, 2 : 도화지를 가로로 2등분, 세로로 2등분.
  # 각각의 영역에 플롯을 하나씩 배치하여 총 4개의 그래프를 그림
  # sharex='col' : 같은 컬럼에 위치한 그래프는 x축을 공유함

for col, ax in zip(trend.columns.tolist(), axes.ravel()):
    # trend.columns.tolist() : trend의 칼럼들 numpy array -> list
    # axes.ravel() : axes의 리스트를 1차원으로 배열시킴
    ax.set_title(col)  # 서브플롯 제목을 나타내라
    ax.axes.xaxis.set_visible(True)   # 서브플롯 x축 간격을 보이도록 나타내라
    ax.plot(trend[col])   # trend[col]의 플롯들을 그려라
plt.show()


In [ ]:

type(trend)

In [ ]:
trend.reset_index(inplace=True)
trend


In [ ]:
from sklearn.linear_model import LinearRegression
for num in range(4):
    data = trend[['Year']]
    label = trend['Topic{}'.format(num+1)]
    model = LinearRegression()                          # 선형회귀에 적용함. X(data)는 연도, Y(label)은 해당연도의 Topic의 비중임.
    model = model.fit(data, label)
    predict = model.predict([[2019.0]])     # model.predict([[2023.0]])
    print("2019, Topic{} : ".format(num+1), predict)    # 2023년에 가장 핫한 토픽은 Topic4(스스로학습법과 육아의 효능관계)로 예상함.   Main 발표 끝 -


In [ ]:

def power(x):
    return x * x
a = power(2)
print(a)

In [ ]:

def power(x):
    return x * x
a = map(power, [1,2,3])  # map함수는 리스트의 요소를 함수에 넣고 리턴된 값으로 새로운 리스트를 구성하는 함수
print(list(a))

In [ ]:

b = map(lambda x: x * x, [1,2,3])
# lambda는 함수를 def로 정의(함수이름 필요함)하지 않고 간단하게 함수(함수이름 필요없음)를 정의하는 것
print(list(b))


In [1]:

# 람다 함수
# x = 2023-07-31
a=map(lambda x: x[:4], ["2023-07-31"])
print(list(a))

['2023']
